In [1]:
import logging
import csv
import os
from datetime import datetime
import pymongo
class mongodb:
    
    def __init__(self):
        #Initializing log file
        self.log_filename = str(datetime.now()).replace(":","_")+"_logfile"+".log"
        logging.basicConfig(filename=self.log_filename,
                    format='%(asctime)s %(name)s - %(levelname)s - %(message)s',
                    filemode='w')
        self.logger = logging.getLogger()
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.setLevel(logging.DEBUG)
    
    def read_csv(self,filepath):
        """
        This method reads the given csv file ann returns in the form of list of dictionary values 
        in - filepath
        out - list of dictionay data
        
        """
        try:
            if os.path.exists(filepath):
                list_dict = []
                self.logger.info("Starting to read given csv file")
                dict_from_csv = {}

                with open(filepath, mode='r') as f:
                    reader = csv.reader(f)
                    fields = next(reader)
                    fields = fields[0].split(";")

                    for i in reader:
                        list_dict.append(dict(zip(fields,i)))
                self.logger.info("Input csv file read successfully")
                return list_dict     

            else:
                self.logger.error("### File does not exist ###")
        except Exception as e:
            self.logger.error("### Exception occured ###"+str(e))
    
    def connect_to_mongodb(self,connectionstring,dbname,tablename):
        """
        This method takes connection string , dbname and collection name for mongo db and established connection
        in - string to connect to mongo db, db name, table name
        out - connection parameter to collection 
        
        """
        try:
            client = pymongo.MongoClient(connectionstring)
            self.logger.info("Mongo DB connection establiblished successfully")
            db = client[dbname]
            collection = db[tablename]
            self.logger.info("Connection to database established")
            return collection
        except Exception as e:
            self.logger.error("### Exception occured connecting to MongoDB ###"+str(e))
    
    def insert_data(self,collection,list_data):
        """
        This method takes collection name and list of dictionary data and inserts into the mongo db table
        in - collection name, list of data to be inserted in dictionary format
        out - None
        
        """
        try:
            collection.insert_many(list_data)
            self.logger.info(str(len(list_data))+" records inserted successfully")
        except Exception as e:
            self.logger.error("### Exception occured while inserting data into Mongodb collections ###"+str(e))
    
    def find_all_data(self,collection):
        """
        This method takes collection and returns all the table data in list format
        in - collection name
        out - list of table data
        
        """
        try:
            l = [i for i in collection.find()]
            self.logger.info("Read the table data successfully")
            return l
        except Exception as e:
            self.logger.error("### Exception occured while reading table data ###"+str(e))
            
    def update_one_data(self,collection,dict_tofind,dict_toupdate):
        """
        This method upates 1 data in the table
        in - collection name, json string to be updated
        out - None
        """
        try:
            collection.find_one_and_update(dict_tofind,dict_toupdate)
            self.logger.info("1 Record updated successfully")
        except Exception as e:
            self.logger.error("### Exception occured while updating table data ###"+str(e))
    
    def update_many_records(self,collection,dict_find,dict_toupdate):
        """
        This method updates bulk records
        in - collection, json string to be updated
        out - Number of records
        """
        try:
            collection.update_many(dict_find,dict_toupdate)
            self.logger.info(" Records updated successfully")
        except Exception as e:
            self.logger.error("### Exception occured while updating table data ###"+str(e))
    
    def delete_records(self,collection,dict_tofind):
        """
        THis method deletes records
        in - collection, dict_to find - to find records and delete
        out - None
        """
        try:
            collection.delete_many(dict_tofind)
            self.logger.info("Records deleted successfully")
        except Exception as e:
            self.logger.error("### Exception occured while deleting table data ###"+str(e))
        
        
        
#create object
m = mongodb()
#Assign db and table values
connectionstring = "mongodb+srv://<username>:<password>.he3fu.mongodb.net/myFirstDatabase?retryWrites=true&w=majority"
dbname = "DB_Test"
tablename = "carbon_nanotubes"
#Establish database connection
collection = m.connect_to_mongodb(connectionstring,dbname,tablename)
#Read csv data set
data = m.read_csv(r"C:\Users\username\Documents\Data science\Datasets\carbon_nanotubes.csv")
#Insert csv dataset into table
m.insert_data(collection,data)
#Read all the data
l = m.find_all_data(collection)
#prin6(l)
#Update many records
dict_tofind = {'Chiral indice m': '679005;0'}
dict_toupdate = {"$set":{'Chiral indice m': '679007;0'}}
m.update_many_records(collection,dict_tofind,dict_toupdate)
#Deleting records
dict_todelete = {'Chiral indice m': '679007;0'}
m.delete_records(collection,dict_todelete)

        